In [24]:
import keras.backend as K
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt


def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def load_dataset():
    train_dataset = h5py.File('train_happy.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_happy.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes



In [25]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [26]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


In [27]:

def HappyModel(input_shape):
    

   # Define the input placeholder as a tensor with shape input_shape. 
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X)
    
    # CONV -> RELU Block applied to X
    X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')

    
    return model

In [29]:

happyModel = HappyModel(X_train.shape[1:])


In [30]:

happyModel.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["accuracy"])


In [31]:

happyModel.fit( x= X_train, y = Y_train, epochs = 40, batch_size = 32)


Instructions for updating:
Use tf.cast instead.
Epoch 1/40
600/600 [==============================] - 67s 111ms/step - loss: 0.1889 - acc: 0.7450
Epoch 2/40
600/600 [==============================] - 64s 106ms/step - loss: 0.0846 - acc: 0.8983
Epoch 3/40
600/600 [==============================] - 64s 107ms/step - loss: 0.0570 - acc: 0.9333
Epoch 4/40
600/600 [==============================] - 65s 108ms/step - loss: 0.0529 - acc: 0.9300
Epoch 5/40
600/600 [==============================] - 65s 109ms/step - loss: 0.0378 - acc: 0.9567
Epoch 6/40
600/600 [==============================] - 66s 111ms/step - loss: 0.0274 - acc: 0.9683
Epoch 7/40
600/600 [==============================] - 65s 108ms/step - loss: 0.0280 - acc: 0.9683
Epoch 8/40
600/600 [==============================] - 64s 107ms/step - loss: 0.0254 - acc: 0.9750
Epoch 9/40
600/600 [==============================] - 64s 106ms/step - loss: 0.0245 - acc: 0.9700
Epoch 10/40
600/600 [==============================] - 63s 106ms/step 

In [33]:

preds = happyModel.evaluate(x = X_test, y = Y_test)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

150/150 [==============================] - 7s 49ms/step

Loss = 0.020494073120256265
Test Accuracy = 0.98


In [ ]:
# Image testing can be done here.
img_path = 'enter image path'
img = image.load_img(img_path, target_size=(64, 64))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(happyModel.predict(x))